In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [20]:
data = pd.read_csv('C:/Jupyter/TESIS/instancias/real/testInstance_Amtrak.csv', delimiter=';')
data.drop(columns=['Unnamed: 0'], inplace=True)

In [21]:
start_estation = 'NYP'

origin = data['Origin'].unique().tolist()
destin = data['Destination'].unique().tolist()
periodo = sorted(data['DBD'].unique().tolist(), reverse=True)

stations = destin + [i for i in origin if i not in destin]

anonymo_stat = {i: data[data['Market'] == start_estation + '-' + i]['NbLegs'].unique()[0] + 1 if i != start_estation else 1 for i in stations }
data.replace(anonymo_stat, inplace=True)
origin_cor = sorted(data['Origin'].unique().tolist())
destin_cor = sorted(data['Destination'].unique().tolist())

In [22]:
data['Market'] = data['Origin'].astype('str') +'-'+ data['Destination'].astype('str')
oridest = data['Market'].unique().tolist()
oridest = sorted([(int(i.split('-')[0]), int(i.split('-')[1])) for i in oridest])

jerar_comer_class = {'Y': '1-1','YA': '1-2','YB': '1-3','YC': '1-4','YD': '1-5','YE': '1-6','YF': '1-7','YG': '1-8','JY': '2-1','JA': '2-2','JB': '2-3','JC': '2-4','JD': '2-5','JE': '2-6','JF': '2-7','JG': '2-8'}
data['CommercialClass'].replace(jerar_comer_class, inplace=True)

data[['Vagon','Clase']] = data['CommercialClass'].str.split('-', expand=True).astype('int')
vagones = data['Vagon'].unique().tolist()
clases = {v: sorted(data[data['Vagon']==v]['Clase'].unique().tolist()) for v in vagones}

In [23]:
demanda = pd.pivot_table(data, values=['Bookings'], index=['Origin','Destination','Vagon','Clase', 'DBD'], aggfunc={'Bookings':'sum'})
demanda.reset_index(inplace=True)

preco = pd.pivot_table(data, values=['Revenue'], index=['Origin','Destination','Vagon','Clase'], aggfunc={'Revenue':'max'})
preco.reset_index(inplace=True)

(65, 5)

In [24]:
def fill_data(oridest, vagones, clases, records_to_correct, periodo=0): #preencha os dados
     
    combin_dem = []
    for v in vagones:
        combin_dem += list(product([v], clases[v]))

    try:
        columnas = ['trecho', 'vagon_clase', 'DBD']
        df = pd.DataFrame(list(product(oridest, combin_dem, periodo)), columns=columnas)
    except:
        columnas = ['trecho', 'vagon_clase']
        df = pd.DataFrame(list(product(oridest, combin_dem)), columns=columnas)

    df[['Origin', 'Destination']] = pd.DataFrame(df['trecho'].tolist(), index=df.index)
    df[['Vagon', 'Clase']] = pd.DataFrame(df['vagon_clase'].tolist(), index=df.index)
    df = df.drop(columns=['trecho', 'vagon_clase'])

    try:
        df = df[['Origin', 'Destination', 'Vagon', 'Clase', 'DBD']]
        df['Bookings'] = 0
        df['clave_completa'] = df['Origin'].astype(str) + '_' + df['Destination'].astype(str) + '_' + df['Vagon'].astype(str) + '_' + df['Clase'].astype(str) + '_' + df['DBD'].astype(str)
        records_to_correct['clave_completa'] = records_to_correct['Origin'].astype(str) + '_' + records_to_correct['Destination'].astype(str) + '_' + records_to_correct['Vagon'].astype(str) + '_' + records_to_correct['Clase'].astype(str) + '_' + records_to_correct['DBD'].astype(str)
    except:
        df['Revenue'] = 0
        df['clave_completa'] = df['Origin'].astype(str) + '_' + df['Destination'].astype(str) + '_' + df['Vagon'].astype(str) + '_' + df['Clase'].astype(str)
        records_to_correct['clave_completa'] = records_to_correct['Origin'].astype(str) + '_' + records_to_correct['Destination'].astype(str) + '_' + records_to_correct['Vagon'].astype(str) + '_' + records_to_correct['Clase'].astype(str)

    new_records = df[~df['clave_completa'].isin(records_to_correct['clave_completa'])]
    records_to_correct = pd.concat([records_to_correct, new_records], ignore_index=True)

    records_to_correct.drop('clave_completa', axis=1, inplace=True)
        
    return records_to_correct


preco = fill_data(oridest,vagones,clases, preco)
demanda = fill_data(oridest,vagones,clases, demanda, periodo)

In [26]:
preco = preco.sort_values(by=['Origin', 'Destination', 'Vagon', 'Revenue'], ascending=[True, True, True, False])

dem_cor = demanda.set_index(['Origin', 'Destination', 'Vagon', 'Clase', 'DBD'])['Bookings'].to_dict()
preco_cor = preco.set_index(['Origin', 'Destination', 'Vagon', 'Clase'])['Revenue'].to_dict()

In [27]:
data.to_excel('data.xlsx', index=False)

In [28]:
# def mode_clase(x):
#     b = x[0]
#     return b

# def sub_clase(x):
#     if len(x)>=2:
#         c = x[1]
#     else:
#         c = ''
#     return c

# data['Vagon'] = data['CommercialClass'].apply(mode_clase)
# vagones = data['Vagon'].unique().tolist()
# data['Vagon'].replace({j:i+1 for i,j in enumerate(vagones)}, inplace= True)

# data['sub_clas'] = data['CommercialClass'].apply(sub_clase)
# sub_clase = sorted(data['sub_clas'].unique().tolist())
# data['sub_clas'].replace({j:i+1 for i,j in enumerate(sub_clase)}, inplace= True)

# vagones_1 = data['Vagon'].unique().tolist()
# clases = data['sub_clas'].unique().tolist()


# for i,j in oridest:
#     for v in vagones:
#         for k in clases[v]:
#                 for t in periodo:
#                     if demanda.loc[(demanda.Origin == i) & (demanda.Destination == j) & (demanda.Vagon == v) & (demanda.Clase == k) & (demanda.DBD == t)]['Bookings'].count() == 0:
#                         a = {'Origin':i,'Destination':j,'Vagon':v,'Clase':k, 'DBD':t,'Bookings':0}
#                         demanda.loc[len(demanda)] = a


# for i,j in oridest:
#     for v in vagones:
#         for k in clases[v]:
#                 if preco.loc[(preco.Origin == i) & (preco.Destination == j) & (preco.Vagon == v) & (preco.Clase == k)]['Revenue'].count() == 0:
#                     a = {'Origin':i,'Destination':j,'Vagon':v,'Clase':k,'Revenue':0}
#                     preco.loc[len(preco)] = a
